# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.

In [24]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[4730, 4728, 4729]


In [44]:
import requests
import json

# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

def get_user(id):
    response = requests.get(f'{sdw2023_api_url}/users/{id}', headers={"content-type":"application/json;charset=ISO-8859-1"})
    #response.encoding = response.apparent_encoding
    #response_encoding = response.json()
    #print(response_encoding)
    return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 4730,
    "name": "Fl\u00e1vio Macedo",
    "account": {
      "id": 5009,
      "number": "111111-3",
      "agency": "11111",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 4593,
      "number": "**** **** **** 9286",
      "limit": 1000.0
    },
    "features": [
      {
        "id": 1456,
        "icon": "string",
        "description": "string"
      }
    ],
    "news": [
      {
        "id": 8763,
        "icon": "string",
        "description": "string"
      }
    ]
  },
  {
    "id": 4728,
    "name": "Gustavo Alcantara",
    "account": {
      "id": 5007,
      "number": "111111-1",
      "agency": "11111",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 4591,
      "number": "**** **** **** 9284",
      "limit": 1000.0
    },
    "features": [
      {
        "id": 1454,
        "icon": "string",
        "description": "string"
      }
    ],
    "news": [
      {
        "id": 8754,
        "icon": 

In [68]:
import os
import openai

from dotenv import load_dotenv

load_dotenv()

# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.

openai_api_key = os.getenv('API_KEY')

openai.api_key = openai_api_key

def generate_ai_news(user):
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {
            "role": "system",
            "content": "Você é um especialista em markting bancário."
        },
        {
            "role": "user",
            "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
        }
      ]
    )
    return completion.choices[0].message.content.strip('\"')

for user in users:
  try:
    news = generate_ai_news(user)
  except:
    news = "A geração da mensagem falhou por conta da falha de comunicação com o ChatGPT. Isso pode ter ocorrido por limite de requisições com a API."
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })


A geração da mensagem falhou por conta da falha de comunicação com o ChatGPT. Isso pode ter ocorrido por limite de requisições com a API.
A geração da mensagem falhou por conta da falha de comunicação com o ChatGPT. Isso pode ter ocorrido por limite de requisições com a API.
A geração da mensagem falhou por conta da falha de comunicação com o ChatGPT. Isso pode ter ocorrido por limite de requisições com a API.
